In [1]:
import os
import sys
import json
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.optim import lr_scheduler

from opts import parse_opts
from model import generate_model
from mean import get_mean, get_std
from spatial_transforms import (
    Compose, Normalize, Scale, CenterCrop, CornerCrop, MultiScaleCornerCrop,
    MultiScaleRandomCrop, RandomHorizontalFlip, ToTensor)
from temporal_transforms import LoopPadding, TemporalRandomCrop
from target_transforms import ClassLabel, VideoID
from target_transforms import Compose as TargetCompose
from dataset import get_training_set, get_validation_set, get_test_set
from utils import Logger
from train import train_epoch
from validation import val_epoch
import test
import collections
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from joblib import dump, load
from sklearn import preprocessing
from scipy import stats
from sklearn.metrics import accuracy_score

from models import *

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='2,3'

### Load pretrained model

In [3]:
from models.resnext import get_fine_tuning_parameters
from models import resnext

# construct model architecture
model_name = 'resnext-101-kinetics'
model = resnext.resnet101(
            num_classes=9,
            shortcut_type='B',
            cardinality=32,
            sample_size=112,
            sample_duration=16)

/home/qq/CSCE689/3D-ResNets-PyTorch/models/resnext.py:121: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [4]:
# check model structure
model

ResNeXt(
  (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResNeXtBottleneck(
      (conv1): Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), groups=32, bias=False)
      (bn2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(128, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequentia

In [5]:
# load pretrained weight
model = model.cuda()
# wrap the current model again in nn.DataParallel / or we can just remove the .module keys.
model = nn.DataParallel(model, device_ids=None)
# filter out unnecessary keys
pretrain = torch.load('/data/qq/CSCE689/pretrain/'+model_name+'.pth')
pretrain_dict = pretrain['state_dict']
# pretrain_dict_filter = {k: v for k, v in pretrain_dict.items() if k in model_dict}

pretrain_dict.pop('module.fc.weight')
pretrain_dict.pop('module.fc.bias')
model_dict = model.state_dict()
model_dict.update(pretrain_dict) 
model.load_state_dict(model_dict)



<All keys matched successfully>

In [6]:
# summarize model input and output
from torchsummary import summary
summary(model, (3, 16, 112, 112))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 64, 16, 56, 56]          65,856
       BatchNorm3d-2       [-1, 64, 16, 56, 56]             128
              ReLU-3       [-1, 64, 16, 56, 56]               0
            Conv3d-4       [-1, 64, 16, 56, 56]          65,856
         MaxPool3d-5        [-1, 64, 8, 28, 28]               0
            Conv3d-6       [-1, 128, 8, 28, 28]           8,192
       BatchNorm3d-7       [-1, 64, 16, 56, 56]             128
              ReLU-8       [-1, 64, 16, 56, 56]               0
       BatchNorm3d-9       [-1, 128, 8, 28, 28]             256
        MaxPool3d-10        [-1, 64, 8, 28, 28]               0
           Conv3d-11       [-1, 128, 8, 28, 28]           8,192
             ReLU-12       [-1, 128, 8, 28, 28]               0
      BatchNorm3d-13       [-1, 128, 8, 28, 28]             256
             ReLU-14       [-1, 128, 8,

In [7]:
# load preprocessed video frames and annotation
from datasets.ucf101 import UCF101
root_path = '/data/qq/CSCE689/video/'
video_path = root_path + 'UCF-music/'  # 'UCF-101-jpg/' 
annotation_path = root_path+'ucfTrainTestlist/ucf101_01_music.json'

In [8]:
## extract music annotation only
# import json

# with open(annotation_path, 'r') as data_file:
#         qq = json.load(data_file)

# labels = []
# for l in qq['labels']:
#     if l[:7] == 'Playing':
#         labels.append(l)
        
# database = {}
# for data in qq['database']:
#     if data[:9] == 'v_Playing':
#         database[data] = qq['database'][data]

# qq_music = {}
# qq_music['labels'] = labels
# qq_music['database'] = database
# with open(root_path+'ucfTrainTestlist/ucf101_03_music.json', 'w') as fp:
#     json.dump(qq_music, fp)

In [9]:
sample_size = 112 # res3d
sample_duration = 16 # for res3d
norm_value = 1
mean = get_mean(norm_value, dataset='kinetics')
std = get_std(norm_value)
norm_method = Normalize(mean, [1,1,1])

batch_size = 32
n_threads = 6

spatial_transform = Compose([
    Scale(sample_size),
    CornerCrop(sample_size, 'c'),
    ToTensor(norm_value), norm_method
])

temporal_transform = LoopPadding(sample_duration)
target_transform = ClassLabel() # VideoID()

# get training data
training_data = UCF101(
    video_path,
    annotation_path,
    'training',
    0,
    spatial_transform=spatial_transform,
    temporal_transform=temporal_transform,
    target_transform=target_transform,
    sample_duration=16)

# wrap training data
train_loader = torch.utils.data.DataLoader(
    training_data,
    batch_size=batch_size,
    shuffle=False,
    num_workers=n_threads,
    pin_memory=False) # True

# get validation data
val_data = UCF101(
    video_path,
    annotation_path,
    'validation',
    0,
    spatial_transform=spatial_transform,
    temporal_transform=temporal_transform,
    target_transform=target_transform,
    sample_duration=16)

# wrap validation data
val_loader = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=False,
    num_workers=n_threads,
    pin_memory=False) # True


dataset loading [0/911]
dataset loading [0/356]


### Fine-tune the pretrained model on the targe UCF-101 dataset

In [10]:
# set hyperparameters 

import easydict
opt = easydict.EasyDict({
    "root_path": '',
    "video_path": 'video_kinetics_jpg',
    "annotation_path": 'kinetics.json',
    "result_path": 'results',
    "dataset": 'ucf101-music', # 'ucf101',
    "n_classes": 9, # 101, 
    "n_finetune_classes": 9, # 101,
    "sample_size": 112,
    "sample_duration": 16,
    "initial_scale": 1.0,
    "n_scales": 5,
    "scale_step": 0.84089641525,
    "train_crop": 'corner',
    "learning_rate": 0.1,
    "momentum": 0.9,
    "dampening": 0.9,
    "weight_decay": 0.001,
    "mean_dataset": 'activitynet',
    "no_mean_norm": False,
    "std_norm": False,
    "nesterov": False,
    "optimizer": 'sgd',
    "lr_patience": 10,
    "batch_size": 32,
    "n_epochs": 20,
    "begin_epoch": 1,
    "n_val_samples": 3,
    "resume_path": '',
    "pretrain_path": '',
    "ft_begin_index": 5,
    "no_train": False,
    "no_val": False,
    "test": False,
    "test_subset": 'val',
    "scale_in_test": 1.0,
    "crop_position_in_test": 'c',
    "no_softmax_in_test": False,
    "no_cuda": False,
    "n_threads": 4,
    "checkpoint": 2,
    "no_hflip": False,
    "norm_value": 1,
    "model": 'resnet',
    "model_depth": 101,
    "resnet_shortcut": 'B',
    "wide_resnet_k": 2,
    "resnext_cardinality": 32,
    "manual_seed": 1
})
opt.arch = '{}-{}'.format(opt.model, opt.model_depth)

In [11]:
criterion = nn.CrossEntropyLoss()
if not opt.no_cuda:
    criterion = criterion.cuda()
    
    
from models.resnext import get_fine_tuning_parameters

# get fine-tune parameters (we fine-tune all of them)
# parameters = model.parameters()
parameters = get_fine_tuning_parameters(model, opt.ft_begin_index)

optimizer = optim.SGD(
            parameters,
            lr=opt.learning_rate,
            momentum=opt.momentum,
            dampening=opt.dampening,
            weight_decay=opt.weight_decay,
            nesterov=opt.nesterov)

scheduler = lr_scheduler.ReduceLROnPlateau(
            optimizer, 'min', patience=opt.lr_patience)

train_logger = Logger(
    os.path.join(opt.result_path, 'train.log'),
    ['epoch', 'loss', 'acc', 'lr'])
train_batch_logger = Logger(
    os.path.join(opt.result_path, 'train_batch.log'),
    ['epoch', 'batch', 'iter', 'loss', 'acc', 'lr'])
val_logger = Logger(
            os.path.join(opt.result_path, 'val.log'), ['epoch', 'loss', 'acc'])

for i in range(opt.begin_epoch, opt.n_epochs + 1):

    train_epoch(i, train_loader, model, criterion, optimizer, 
                opt, train_logger, train_batch_logger)
    
    validation_loss = val_epoch(i, val_loader, model, criterion, 
                                opt, val_logger)
    
    scheduler.step(validation_loss)

train at epoch 1
Epoch: [1][1/492]	Time 2.110 (2.110)	Data 1.682 (1.682)	Loss 2.1519 (2.1519)	Acc 0.344 (0.344)
Epoch: [1][2/492]	Time 0.627 (1.369)	Data 0.152 (0.917)	Loss 7.3532 (4.7525)	Acc 0.000 (0.172)
Epoch: [1][3/492]	Time 0.480 (1.073)	Data 0.027 (0.620)	Loss 6.9734 (5.4928)	Acc 0.500 (0.281)
Epoch: [1][4/492]	Time 0.475 (0.923)	Data 0.012 (0.468)	Loss 14.5017 (7.7450)	Acc 0.000 (0.211)
Epoch: [1][5/492]	Time 0.467 (0.832)	Data 0.012 (0.377)	Loss 22.0393 (10.6039)	Acc 0.000 (0.169)
Epoch: [1][6/492]	Time 0.465 (0.771)	Data 0.013 (0.316)	Loss 25.5999 (13.1032)	Acc 0.000 (0.141)
Epoch: [1][7/492]	Time 0.480 (0.729)	Data 0.024 (0.275)	Loss 6.6276 (12.1781)	Acc 0.344 (0.170)
Epoch: [1][8/492]	Time 0.580 (0.711)	Data 0.020 (0.243)	Loss 17.3947 (12.8302)	Acc 0.031 (0.152)
Epoch: [1][9/492]	Time 0.484 (0.685)	Data 0.026 (0.219)	Loss 2.4137 (11.6728)	Acc 0.094 (0.146)
Epoch: [1][10/492]	Time 0.464 (0.663)	Data 0.014 (0.198)	Loss 22.4885 (12.7544)	Acc 0.000 (0.131)
Epoch: [1][11/492]	Ti

Epoch: [1][86/492]	Time 0.491 (0.497)	Data 0.014 (0.039)	Loss 5.1166 (7.9273)	Acc 0.344 (0.238)
Epoch: [1][87/492]	Time 0.467 (0.497)	Data 0.013 (0.039)	Loss 7.8371 (7.9263)	Acc 0.188 (0.237)
Epoch: [1][88/492]	Time 0.495 (0.497)	Data 0.022 (0.039)	Loss 11.9674 (7.9722)	Acc 0.031 (0.235)
Epoch: [1][89/492]	Time 0.473 (0.497)	Data 0.014 (0.038)	Loss 7.7958 (7.9702)	Acc 0.062 (0.233)
Epoch: [1][90/492]	Time 0.469 (0.497)	Data 0.014 (0.038)	Loss 16.8336 (8.0687)	Acc 0.000 (0.230)
Epoch: [1][91/492]	Time 0.472 (0.496)	Data 0.017 (0.038)	Loss 5.8340 (8.0442)	Acc 0.031 (0.228)
Epoch: [1][92/492]	Time 0.467 (0.496)	Data 0.014 (0.038)	Loss 4.4453 (8.0050)	Acc 0.188 (0.228)
Epoch: [1][93/492]	Time 0.480 (0.496)	Data 0.022 (0.037)	Loss 12.2043 (8.0502)	Acc 0.125 (0.226)
Epoch: [1][94/492]	Time 0.480 (0.496)	Data 0.021 (0.037)	Loss 1.3350 (7.9788)	Acc 0.562 (0.230)
Epoch: [1][95/492]	Time 0.479 (0.495)	Data 0.022 (0.037)	Loss 0.6899 (7.9020)	Acc 0.750 (0.236)
Epoch: [1][96/492]	Time 0.471 (0.495)

Epoch: [1][171/492]	Time 0.469 (0.488)	Data 0.013 (0.029)	Loss 5.2963 (6.4425)	Acc 0.062 (0.243)
Epoch: [1][172/492]	Time 0.487 (0.488)	Data 0.023 (0.029)	Loss 1.6750 (6.4148)	Acc 0.500 (0.244)
Epoch: [1][173/492]	Time 0.486 (0.488)	Data 0.022 (0.029)	Loss 2.7702 (6.3937)	Acc 0.281 (0.244)
Epoch: [1][174/492]	Time 0.482 (0.488)	Data 0.021 (0.029)	Loss 1.2411 (6.3641)	Acc 0.625 (0.247)
Epoch: [1][175/492]	Time 0.478 (0.488)	Data 0.018 (0.029)	Loss 5.3564 (6.3583)	Acc 0.312 (0.247)
Epoch: [1][176/492]	Time 0.470 (0.488)	Data 0.015 (0.029)	Loss 12.6355 (6.3940)	Acc 0.031 (0.246)
Epoch: [1][177/492]	Time 0.479 (0.488)	Data 0.020 (0.029)	Loss 7.5296 (6.4004)	Acc 0.062 (0.245)
Epoch: [1][178/492]	Time 0.479 (0.488)	Data 0.016 (0.028)	Loss 6.1971 (6.3993)	Acc 0.219 (0.245)
Epoch: [1][179/492]	Time 0.483 (0.488)	Data 0.022 (0.028)	Loss 1.3177 (6.3709)	Acc 0.688 (0.247)
Epoch: [1][180/492]	Time 0.480 (0.488)	Data 0.021 (0.028)	Loss 6.4531 (6.3713)	Acc 0.094 (0.246)
Epoch: [1][181/492]	Time 0.48

Epoch: [1][256/492]	Time 0.484 (0.488)	Data 0.023 (0.026)	Loss 5.6417 (6.1099)	Acc 0.062 (0.252)
Epoch: [1][257/492]	Time 0.494 (0.488)	Data 0.023 (0.026)	Loss 3.6691 (6.1004)	Acc 0.156 (0.252)
Epoch: [1][258/492]	Time 0.485 (0.488)	Data 0.023 (0.026)	Loss 1.7316 (6.0834)	Acc 0.469 (0.253)
Epoch: [1][259/492]	Time 0.473 (0.488)	Data 0.015 (0.026)	Loss 5.2776 (6.0803)	Acc 0.062 (0.252)
Epoch: [1][260/492]	Time 0.474 (0.488)	Data 0.013 (0.026)	Loss 3.8194 (6.0716)	Acc 0.281 (0.252)
Epoch: [1][261/492]	Time 0.480 (0.488)	Data 0.021 (0.026)	Loss 1.6667 (6.0547)	Acc 0.594 (0.254)
Epoch: [1][262/492]	Time 0.485 (0.488)	Data 0.023 (0.026)	Loss 9.8034 (6.0690)	Acc 0.062 (0.253)
Epoch: [1][263/492]	Time 0.473 (0.488)	Data 0.014 (0.026)	Loss 9.6572 (6.0827)	Acc 0.000 (0.252)
Epoch: [1][264/492]	Time 0.475 (0.487)	Data 0.013 (0.026)	Loss 8.3622 (6.0913)	Acc 0.000 (0.251)
Epoch: [1][265/492]	Time 0.488 (0.487)	Data 0.026 (0.026)	Loss 6.8419 (6.0942)	Acc 0.312 (0.251)
Epoch: [1][266/492]	Time 0.505

Epoch: [1][341/492]	Time 0.472 (0.486)	Data 0.014 (0.024)	Loss 3.6095 (5.7489)	Acc 0.156 (0.268)
Epoch: [1][342/492]	Time 0.473 (0.486)	Data 0.014 (0.024)	Loss 1.2686 (5.7358)	Acc 0.594 (0.269)
Epoch: [1][343/492]	Time 0.486 (0.486)	Data 0.023 (0.024)	Loss 1.2850 (5.7228)	Acc 0.688 (0.270)
Epoch: [1][344/492]	Time 0.483 (0.486)	Data 0.021 (0.024)	Loss 8.2598 (5.7302)	Acc 0.406 (0.271)
Epoch: [1][345/492]	Time 0.477 (0.486)	Data 0.014 (0.024)	Loss 1.3467 (5.7175)	Acc 0.594 (0.272)
Epoch: [1][346/492]	Time 0.473 (0.486)	Data 0.014 (0.024)	Loss 3.9947 (5.7125)	Acc 0.281 (0.272)
Epoch: [1][347/492]	Time 0.476 (0.486)	Data 0.014 (0.024)	Loss 0.9296 (5.6987)	Acc 0.594 (0.273)
Epoch: [1][348/492]	Time 0.470 (0.486)	Data 0.014 (0.024)	Loss 6.2452 (5.7003)	Acc 0.062 (0.272)
Epoch: [1][349/492]	Time 0.478 (0.486)	Data 0.016 (0.024)	Loss 4.1354 (5.6958)	Acc 0.156 (0.272)
Epoch: [1][350/492]	Time 0.498 (0.486)	Data 0.018 (0.024)	Loss 3.1875 (5.6886)	Acc 0.250 (0.272)
Epoch: [1][351/492]	Time 0.477

...

Epoch: [20][337/492]	Time 0.474 (0.484)	Data 0.015 (0.021)	Loss 2.0083 (1.3215)	Acc 0.344 (0.590)
Epoch: [20][338/492]	Time 0.476 (0.484)	Data 0.015 (0.021)	Loss 0.9059 (1.3202)	Acc 0.719 (0.590)
Epoch: [20][339/492]	Time 0.478 (0.484)	Data 0.016 (0.021)	Loss 0.1969 (1.3169)	Acc 1.000 (0.592)
Epoch: [20][340/492]	Time 0.474 (0.484)	Data 0.015 (0.021)	Loss 0.8350 (1.3155)	Acc 0.719 (0.592)
Epoch: [20][341/492]	Time 0.477 (0.484)	Data 0.015 (0.021)	Loss 2.3573 (1.3186)	Acc 0.344 (0.591)
Epoch: [20][342/492]	Time 0.474 (0.484)	Data 0.016 (0.021)	Loss 0.8014 (1.3170)	Acc 0.750 (0.592)
Epoch: [20][343/492]	Time 0.479 (0.484)	Data 0.015 (0.021)	Loss 1.0244 (1.3162)	Acc 0.688 (0.592)
Epoch: [20][344/492]	Time 0.475 (0.484)	Data 0.015 (0.021)	Loss 0.8241 (1.3148)	Acc 0.812 (0.593)
Epoch: [20][345/492]	Time 0.477 (0.483)	Data 0.015 (0.021)	Loss 1.0426 (1.3140)	Acc 0.656 (0.593)
Epoch: [20][346/492]	Time 0.475 (0.483)	Data 0.016 (0.021)	Loss 1.5364 (1.3146)	Acc 0.438 (0.592)
Epoch: [20][347/492]

Epoch: [20][421/492]	Time 0.478 (0.482)	Data 0.015 (0.020)	Loss 0.7981 (1.3236)	Acc 0.688 (0.591)
Epoch: [20][422/492]	Time 0.476 (0.482)	Data 0.015 (0.020)	Loss 0.5998 (1.3219)	Acc 0.781 (0.591)
Epoch: [20][423/492]	Time 0.486 (0.482)	Data 0.022 (0.020)	Loss 1.0205 (1.3212)	Acc 0.719 (0.592)
Epoch: [20][424/492]	Time 0.475 (0.482)	Data 0.015 (0.020)	Loss 1.7508 (1.3222)	Acc 0.469 (0.591)
Epoch: [20][425/492]	Time 0.474 (0.482)	Data 0.015 (0.020)	Loss 1.7345 (1.3232)	Acc 0.344 (0.591)
Epoch: [20][426/492]	Time 0.479 (0.482)	Data 0.015 (0.020)	Loss 1.9170 (1.3246)	Acc 0.281 (0.590)
Epoch: [20][427/492]	Time 0.473 (0.482)	Data 0.015 (0.020)	Loss 2.1565 (1.3265)	Acc 0.156 (0.589)
Epoch: [20][428/492]	Time 0.473 (0.482)	Data 0.014 (0.020)	Loss 1.0208 (1.3258)	Acc 0.656 (0.589)
Epoch: [20][429/492]	Time 0.475 (0.482)	Data 0.015 (0.020)	Loss 0.5506 (1.3240)	Acc 0.875 (0.590)
Epoch: [20][430/492]	Time 0.477 (0.482)	Data 0.015 (0.020)	Loss 0.8187 (1.3228)	Acc 0.906 (0.590)
Epoch: [20][431/492]

Epoch: [20][13/186]	Time 0.496 (0.344)	Data 0.372 (0.233)	Loss 0.6065 (0.6854)	Acc 0.844 (0.764)
Epoch: [20][14/186]	Time 0.140 (0.329)	Data 0.031 (0.219)	Loss 1.7482 (0.7613)	Acc 0.500 (0.746)
Epoch: [20][15/186]	Time 0.155 (0.318)	Data 0.048 (0.207)	Loss 0.0588 (0.7145)	Acc 1.000 (0.763)
Epoch: [20][16/186]	Time 0.164 (0.308)	Data 0.058 (0.198)	Loss 0.0800 (0.6748)	Acc 1.000 (0.777)
Epoch: [20][17/186]	Time 0.156 (0.299)	Data 0.056 (0.190)	Loss 0.0717 (0.6393)	Acc 1.000 (0.790)
Epoch: [20][18/186]	Time 0.168 (0.292)	Data 0.050 (0.182)	Loss 0.5780 (0.6359)	Acc 0.812 (0.792)
Epoch: [20][19/186]	Time 0.417 (0.298)	Data 0.313 (0.189)	Loss 0.0293 (0.6040)	Acc 1.000 (0.803)
Epoch: [20][20/186]	Time 0.166 (0.292)	Data 0.049 (0.182)	Loss 0.1151 (0.5795)	Acc 0.969 (0.811)
Epoch: [20][21/186]	Time 0.138 (0.284)	Data 0.031 (0.175)	Loss 0.8839 (0.5940)	Acc 0.688 (0.805)
Epoch: [20][22/186]	Time 0.179 (0.280)	Data 0.058 (0.169)	Loss 0.0094 (0.5675)	Acc 1.000 (0.814)
Epoch: [20][23/186]	Time 0.138

Epoch: [20][98/186]	Time 0.511 (0.222)	Data 0.407 (0.113)	Loss 0.4792 (0.6165)	Acc 0.906 (0.793)
Epoch: [20][99/186]	Time 0.153 (0.221)	Data 0.051 (0.113)	Loss 0.4897 (0.6153)	Acc 0.812 (0.794)
Epoch: [20][100/186]	Time 0.152 (0.220)	Data 0.049 (0.112)	Loss 0.5423 (0.6145)	Acc 0.875 (0.794)
Epoch: [20][101/186]	Time 0.145 (0.219)	Data 0.049 (0.111)	Loss 0.1678 (0.6101)	Acc 1.000 (0.796)
Epoch: [20][102/186]	Time 0.151 (0.219)	Data 0.048 (0.111)	Loss 0.4937 (0.6090)	Acc 0.781 (0.796)
Epoch: [20][103/186]	Time 0.165 (0.218)	Data 0.052 (0.110)	Loss 0.7548 (0.6104)	Acc 0.781 (0.796)
Epoch: [20][104/186]	Time 0.439 (0.220)	Data 0.333 (0.112)	Loss 0.2522 (0.6069)	Acc 0.938 (0.797)
Epoch: [20][105/186]	Time 0.162 (0.220)	Data 0.044 (0.112)	Loss 1.8929 (0.6192)	Acc 0.250 (0.792)
Epoch: [20][106/186]	Time 0.134 (0.219)	Data 0.032 (0.111)	Loss 1.2615 (0.6253)	Acc 0.594 (0.790)
Epoch: [20][107/186]	Time 0.143 (0.218)	Data 0.049 (0.110)	Loss 0.4722 (0.6238)	Acc 0.906 (0.791)
Epoch: [20][108/186]	T

Epoch: [20][182/186]	Time 0.263 (0.212)	Data 0.163 (0.106)	Loss 0.3273 (0.5903)	Acc 0.906 (0.798)
Epoch: [20][183/186]	Time 0.365 (0.213)	Data 0.260 (0.107)	Loss 0.7376 (0.5911)	Acc 0.625 (0.797)
Epoch: [20][184/186]	Time 0.151 (0.212)	Data 0.042 (0.106)	Loss 0.2592 (0.5893)	Acc 1.000 (0.799)
Epoch: [20][185/186]	Time 0.139 (0.212)	Data 0.041 (0.106)	Loss 0.0074 (0.5862)	Acc 1.000 (0.800)
Epoch: [20][186/186]	Time 0.149 (0.212)	Data 0.050 (0.106)	Loss 0.2403 (0.5844)	Acc 0.967 (0.801)
